# Radar Mosiac Plotting

This notebook is an example of downloading, gridding, and plotting radar mosiacs

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import requests

from datetime import datetime, timedelta
from netCDF4 import Dataset

import glob
import os

import pyart

import pandas as pd

import numpy as np

import zipfile

import shutil

import mmmpy

In [ ]:
#set date and location bounds
#set time to plot (no leading zeros)
year  = 2021
month = 12
day   = 11
hour  = 0
minute = 0
    
lat_min = 40
lat_max = 50
lon_min = -105
lon_max = -85

dt = datetime(year,month,day,hour,minute)
url = 'https://mrms.agron.iastate.edu/' + dt.strftime('%Y/%m/%d/%Y%m%d%H') + '.zip'
print(dt.hour)

In [ ]:
#remove any old data
try:
    shutil.rmtree('./mrms_data')
except:
    pass

os.mkdir('./mrms_data')

dt = datetime(year,month,day,hour,minute)

#download file and extract
outfile =  './mrms_data/' + dt.strftime('%Y%m%d%H') + '.zip'
url = 'https://mrms.agron.iastate.edu/' + dt.strftime('%Y/%m/%d/%Y%m%d%H') + '.zip'

r = requests.get(url,allow_redirects=True)
open(outfile,'wb').write(r.content)
with zipfile.ZipFile(outfile,'r') as zip_ref:
    zip_ref.extractall('./mrms_data/')


In [ ]:
path = './mrms_data/*/CONUS/MergedBaseReflectivity/*'

dat_file = sorted(glob.glob(path))[0]
mosaic = mmmpy.MosaicTile(dat_file)

print(datetime.utcfromtimestamp(mosaic.Time))

In [ ]:
ext = [lon_min, lon_max, lat_min, lat_max]
fig = plt.figure(figsize=(8, 8), dpi=200)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)
ax.set_extent(ext)

# Create a feature for States/Admin 1 regions at 1:10m from Natural Earth
ax.add_feature(cfeature.STATES,edgecolor='k',linewidth=0.7)
ax.add_feature(cfeature.COASTLINE,edgecolor='k',linewidth=1.0)

mosaic.get_comp()
valmask = np.ma.masked_where(mosaic.mrefl3d_comp <= 0, mosaic.mrefl3d_comp)
cs = plt.pcolormesh(mosaic.Longitude, mosaic.Latitude, valmask, vmin=0, vmax=50,
                    cmap='pyart_HomeyerRainbow', transform=projection)
plt.colorbar(cs, label='Composite Reflectivity (dBZ)',
             orientation='horizontal', pad=0.05, shrink=0.75, fraction=0.05, aspect=30)
plt.title(dt.strftime('%m/%d/%Y %H:%M UTC'))
outfile_date = dt.strftime('%Y%m%d_%H%M')
fig.tight_layout()
fig.savefig(f'{outfile_date}_mosaic.png',bbox_inches='tight')